In [421]:
from sympy import *
from sympy.solvers.ode.systems import linodesolve

print("set up linear ODE assuming no motor inertia")

m_l, k = symbols('m_l, k') # mass of load, spring stiffness
T1, T2 = symbols('T1, T2') # transmission before and after spring
tau_max, omega_max = symbols("tau_max, omega_max") # stall torque, free speed of motor

c0, c1, c2, c3 = symbols('c:4')
c0 = -k * T1**2 * omega_max / tau_max
c1 = k * T1 * omega_max / (tau_max * T2)
c2 = k * T1 / (T2 * m_l)
c3 = -k / (T2**2 * m_l)

# xdot = Ax + b
A = Matrix([
    [c0, c1, 0],
    [0, 0, 1],
    [c2, c3, 0],
])
b = Matrix([omega_max, 0, 0])

display(Eq(MatrixSymbol('\dot{x}', 3, 1), A*MatrixSymbol('x', 3, 1) + b))

set up linear ODE assuming no motor inertia


Eq(\dot{x}, Matrix([
[omega_max],
[        0],
[        0]]) + Matrix([
[-T1**2*k*omega_max/tau_max, T1*k*omega_max/(T2*tau_max), 0],
[                         0,                           0, 1],
[             T1*k/(T2*m_l),              -k/(T2**2*m_l), 0]])*x)

In [422]:
print("solving linear ODE")

t = symbols('t') # time
linsol = linodesolve(A, t, b=b)

for i in range(3):
    linsol[i] = linsol[i].doit(conds='none')
    linsol[i] = simplify(linsol[i])

dum_syms = []
freesyms = list(linsol[0].free_symbols) + list(linsol[1].free_symbols) + list(linsol[2].free_symbols)
for freesym in freesyms:
    if freesym.name.startswith('Dummy') and not freesym in dum_syms:
        dum_syms.append(freesym)
dum_syms = sorted(dum_syms, key=lambda x: x.name)

a0, a1, a2 = symbols("a:3")
a_syms = (a0, a1, a2)
dum_syms_dict = {dum_sym: a_sym for dum_sym,a_sym in zip(dum_syms, a_syms)}

print(f"replacing {dum_syms} with {a_syms}")
for i in range(3):
    linsol[i] = linsol[i].subs(dum_syms_dict)
    display(linsol[i])

solving linear ODE
replacing [_Dummy_7040, _Dummy_7041, _Dummy_7042] with (a0, a1, a2)


-T1**2*T2**2*m_l*omega_max**2/tau_max - T1*T2*a1*m_l*omega_max*exp(-T1**2*k*omega_max*t/(2*tau_max) - t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/tau_max - T1*T2*a2*m_l*omega_max*exp(-T1**2*k*omega_max*t/(2*tau_max) + t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/tau_max + omega_max*t + a0/(T1*T2)

-T1**3*T2**3*m_l*omega_max**2/tau_max - T1**2*T2**2*a1*m_l*omega_max*exp(-T1**2*k*omega_max*t/(2*tau_max) - t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(2*tau_max) - T1**2*T2**2*a2*m_l*omega_max*exp(-T1**2*k*omega_max*t/(2*tau_max) + t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(2*tau_max) + T1*T2*omega_max*t + T2*a1*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)*exp(-T1**2*k*omega_max*t/(2*tau_max) - t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(2*k*tau_max) - T2*a2*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)*exp(-T1**2*k*omega_max*t/(2*tau_max) + t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(2*k*tau_max) + a0

T1*T2*omega_max + a1*exp(-T1**2*k*omega_max*t/(2*tau_max) - t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max)) + a2*exp(-T1**2*k*omega_max*t/(2*tau_max) + t*sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))

In [423]:
print("sqrt(radicand) is purely imaginary, so replace it with s*I where s=sqrt(-radicand)")
s = symbols('s')
radicand = T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2 #inside the sqrt
sqrt_dict = {sqrt(radicand): s*I}
s_dict = {s: sqrt(-radicand)} #subs this into final expression to put back radicand

print("replace real part of exponent with r for conciseness")
r = symbols('r')
realexp = T1**2*k*omega_max/(2*tau_max)
realexp_dict = {realexp: r}
r_dict = {r: realexp} #subs this into final expression to put back radicand

print(f"expand (a1, a2) into (a_a ± a_b * i) because they are complex conjugates")
a_a, a_b = symbols("a_a, a_b")
conj_dict = {
    a1:a_a + a_b * I,
    a2:a_a - a_b * I
}
a_syms = (a0, a_a, a_b)


linsol_clean = []
for i in range(3):
    cleaned = linsol[i].subs(sqrt_dict).subs(realexp_dict).subs(conj_expand)
    linsol_clean.append(cleaned)
    display(linsol_clean[i])

sqrt(radicand) is purely imaginary, so replace it with s*I where s=sqrt(-radicand)
replace real part of exponent with r for conciseness
expand (a1, a2) into (a_a ± a_b * i) because they are complex conjugates


-T1**2*T2**2*m_l*omega_max**2/tau_max - T1*T2*m_l*omega_max*(a_a - I*a_b)*exp(-r*t + I*s*t/(2*T2*m_l*tau_max))/tau_max - T1*T2*m_l*omega_max*(a_a + I*a_b)*exp(-r*t - I*s*t/(2*T2*m_l*tau_max))/tau_max + omega_max*t + a0/(T1*T2)

-T1**3*T2**3*m_l*omega_max**2/tau_max - T1**2*T2**2*m_l*omega_max*(a_a - I*a_b)*exp(-r*t + I*s*t/(2*T2*m_l*tau_max))/(2*tau_max) - T1**2*T2**2*m_l*omega_max*(a_a + I*a_b)*exp(-r*t - I*s*t/(2*T2*m_l*tau_max))/(2*tau_max) + T1*T2*omega_max*t - I*T2*s*(a_a - I*a_b)*exp(-r*t + I*s*t/(2*T2*m_l*tau_max))/(2*k*tau_max) + I*T2*s*(a_a + I*a_b)*exp(-r*t - I*s*t/(2*T2*m_l*tau_max))/(2*k*tau_max) + a0

T1*T2*omega_max + (a_a - I*a_b)*exp(-r*t + I*s*t/(2*T2*m_l*tau_max)) + (a_a + I*a_b)*exp(-r*t - I*s*t/(2*T2*m_l*tau_max))

In [424]:
exp_m = exp(-r*t - I*s*t/(2*T2*m_l*tau_max))
exp_p = exp(-r*t + I*s*t/(2*T2*m_l*tau_max))
trigh = -sinh(r*t)+cosh(r*t) # simplifes to exp(-R)

exp_m_simp = exp_m.factor().rewrite(cos).subs(trigh, trigh.simplify())
exp_p_simp = exp_p.factor().rewrite(cos).subs(trigh, trigh.simplify())
exp_dict = {exp_m:exp_m_simp, exp_p:exp_p_simp}

print("replace imaginary part of exponential with trig")
display(Eq(exp_m, exp_m_simp), Eq(exp_m, exp_m_simp))

print("substitute into solution")
for i in range(3):
    linsol_clean[i] = linsol_clean[i].subs(comp_exp_dict).simplify().expand()
    display(linsol_clean[i])

replace imaginary part of exponential with trig


Eq(exp(-r*t - I*s*t/(2*T2*m_l*tau_max)), (-I*sin(s*t/(2*T2*m_l*tau_max)) + cos(s*t/(2*T2*m_l*tau_max)))*exp(-r*t))

Eq(exp(-r*t - I*s*t/(2*T2*m_l*tau_max)), (-I*sin(s*t/(2*T2*m_l*tau_max)) + cos(s*t/(2*T2*m_l*tau_max)))*exp(-r*t))

substitute into solution


-T1**2*T2**2*m_l*omega_max**2/tau_max - 2*T1*T2*a_a*m_l*omega_max*exp(-r*t)*cos(s*t/(2*T2*m_l*tau_max))/tau_max - 2*T1*T2*a_b*m_l*omega_max*exp(-r*t)*sin(s*t/(2*T2*m_l*tau_max))/tau_max + omega_max*t + a0/(T1*T2)

-T1**3*T2**3*m_l*omega_max**2/tau_max - T1**2*T2**2*a_a*m_l*omega_max*exp(-r*t)*cos(s*t/(2*T2*m_l*tau_max))/tau_max - T1**2*T2**2*a_b*m_l*omega_max*exp(-r*t)*sin(s*t/(2*T2*m_l*tau_max))/tau_max + T1*T2*omega_max*t + T2*a_a*s*exp(-r*t)*sin(s*t/(2*T2*m_l*tau_max))/(k*tau_max) - T2*a_b*s*exp(-r*t)*cos(s*t/(2*T2*m_l*tau_max))/(k*tau_max) + a0

T1*T2*omega_max + 2*a_a*exp(-r*t)*cos(s*t/(2*T2*m_l*tau_max)) + 2*a_b*exp(-r*t)*sin(s*t/(2*T2*m_l*tau_max))

In [425]:
print("Represent solution at t=0 in Ax=b form")
mat = linear_eq_to_matrix(linsol_clean, a_syms)
A0 = simplify(mat[0].subs({t:0}))
A0_inv = simplify(A0.inv())
b0 = simplify(mat[1].subs({t:0}))

print("A0:")
display(A0)
print("b0:")
display(b0)

print("check if solution is accurately represented:")
display(Matrix([linsol_clean[i].subs(t,0) for i in range(3)]))

A_dum @ Matrix(a_syms) - b_dum #should be the same as linsol in previous cell

Represent solution at t=0 in Ax=b form
A0:


Matrix([
[1/(T1*T2),     -2*T1*T2*m_l*omega_max/tau_max,                 0],
[        1, -T1**2*T2**2*m_l*omega_max/tau_max, -T2*s/(k*tau_max)],
[        0,                                  2,                 0]])

b0:


Matrix([
[T1**2*T2**2*m_l*omega_max**2/tau_max],
[T1**3*T2**3*m_l*omega_max**2/tau_max],
[                    -T1*T2*omega_max]])

check if solution is accurately represented:


Matrix([
[                   -T1**2*T2**2*m_l*omega_max**2/tau_max - 2*T1*T2*a_a*m_l*omega_max/tau_max + a0/(T1*T2)],
[-T1**3*T2**3*m_l*omega_max**2/tau_max - T1**2*T2**2*a_a*m_l*omega_max/tau_max - T2*a_b*s/(k*tau_max) + a0],
[                                                                                  T1*T2*omega_max + 2*a_a]])

Matrix([
[                   -T1**2*T2**2*m_l*omega_max**2/tau_max - 2*T1*T2*a_a*m_l*omega_max/tau_max + a0/(T1*T2)],
[-T1**3*T2**3*m_l*omega_max**2/tau_max - T1**2*T2**2*a_a*m_l*omega_max/tau_max - T2*a_b*s/(k*tau_max) + a0],
[                                                                                  T1*T2*omega_max + 2*a_a]])

In [426]:
print(f"Solve for {a_syms} using initial conditions")
x_a0, x_l0, v_l0 = symbols('x_a0, x_l0, v_l0')
ics = Matrix([x_a0, x_l0, v_l0])

a_vals = simplify(A_dum_inv * (ics + b_dum))
a_dict = {a_syms[i]: a_vals[i] for i in range(3)}

a_vals

Solve for (a0, a_a, a_b) using initial conditions


Matrix([
[                                                            T1*T2*(T1*T2*m_l*omega_max*v_l0 + tau_max*x_a0)/tau_max],
[                                                                                        -T1*T2*omega_max/2 + v_l0/2],
[k*(-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)/(2*T2*s)]])

In [427]:
print(f"Substitute {a_syms} back and put back full expressions for s and r")

linsol_ics = []
for i in range(3):
    temp = linsol_clean[i].subs(a_dict).subs(s_dict).subs(r_dict)
    # temp = temp.simplify() #actually increases the operations after cse()
    display(temp)
    linsol_ics.append(temp)

Substitute (a0, a_a, a_b) back and put back full expressions for s and r


-T1**2*T2**2*m_l*omega_max**2/tau_max - 2*T1*T2*m_l*omega_max*(-T1*T2*omega_max/2 + v_l0/2)*exp(-T1**2*k*omega_max*t/(2*tau_max))*cos(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/tau_max - T1*k*m_l*omega_max*(-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)*exp(-T1**2*k*omega_max*t/(2*tau_max))*sin(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(tau_max*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)) + omega_max*t + (T1*T2*m_l*omega_max*v_l0 + tau_max*x_a0)/tau_max

-T1**3*T2**3*m_l*omega_max**2/tau_max - T1**2*T2**2*m_l*omega_max*(-T1*T2*omega_max/2 + v_l0/2)*exp(-T1**2*k*omega_max*t/(2*tau_max))*cos(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/tau_max - T1**2*T2*k*m_l*omega_max*(-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)*exp(-T1**2*k*omega_max*t/(2*tau_max))*sin(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(2*tau_max*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)) + T1*T2*omega_max*t + T1*T2*(T1*T2*m_l*omega_max*v_l0 + tau_max*x_a0)/tau_max + T2*(-T1*T2*omega_max/2 + v_l0/2)*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)*exp(-T1**2*k*omega_max*t/(2*tau_max))*sin(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(k*tau_max) - (-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)*exp

T1*T2*omega_max + 2*(-T1*T2*omega_max/2 + v_l0/2)*exp(-T1**2*k*omega_max*t/(2*tau_max))*cos(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max)) + k*(-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)*exp(-T1**2*k*omega_max*t/(2*tau_max))*sin(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(T2*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2))

In [428]:
from sympy.codegen.ast import CodeBlock, Assignment

x_a_code, x_l_code, v_l_code = symbols('x_a_code, x_l_code, v_l_code')

code = CodeBlock(
    Assignment(x_a_code, linsol_ics[0]),
    Assignment(x_l_code, linsol_ics[1]),
    Assignment(v_l_code, linsol_ics[2]),
).cse()

print(f"# {count_ops(code)} operations")

print(pycode(code))

# 89 operations
x0 = omega_max**2
x1 = m_l*x0
x2 = 1/tau_max
x3 = T1**2
x4 = T2**2
x5 = x2*x3*x4
x6 = T1*T2*omega_max
x7 = m_l*v_l0*x6 + tau_max*x_a0
x8 = (1/2)*v_l0 - 1/2*x6
x9 = 1/T2
x10 = math.sqrt(-T1**4*k**2*m_l**2*x0*x4 + 4*k*m_l*tau_max**2)
x11 = (1/2)*x2
x12 = t*x10*x11*x9/m_l
x13 = math.exp(-k*omega_max*t*x11*x3)
x14 = x13*math.cos(x12)
x15 = x14*x8
x16 = 2*x15
x17 = T1**3*T2**3*x1
x18 = 2*T1*T2*tau_max*x_a0 + m_l*omega_max*v_l0*x3*x4 - 2*tau_max*x_l0 - x17
x19 = math.sin(x12)
x20 = x13*x19/x10
x21 = x18*x20
x22 = k*m_l*omega_max
x23 = x11*x18
x_a_code = -T1*x2*x21*x22 - m_l*x16*x2*x6 + omega_max*t - x1*x5 + x2*x7
x_l_code = T1*T2*omega_max*t + T1*T2*x2*x7 - T2*x20*x22*x23*x3 + T2*x10*x13*x19*x2*x8/k - m_l*omega_max*x15*x5 - x14*x23 - x17*x2
v_l_code = k*x21*x9 + x16 + x6


In [429]:
import numpy as np

nums = {
    T1: 0.00025, #0.035 m / 140 rad from TSA test
    T2: 5, #guess for leg movement/string contract
    m_a: 1/2 * 0.020 * 0.010**2, #cylinder MOI 20g 10mm radius
    m_l: 0.5, #mass to push off ground
    k: 3000, # no idea honestly
    tau_max: 0.05, #N*m stall torque
    omega_max: 3000, #about 10,000rpm free speed
}

ics_n = {x_a0:0, x_l0:0, v_l0:0}

for ti in np.linspace(0, 0.5, 10):
    x_a_ti = linsol_ics[0].subs(ics_n).subs(nums).subs(t,ti).n()
    x_l_ti = linsol_ics[1].subs(ics_n).subs(nums).subs(t,ti).n()
    v_l_ti = linsol_ics[2].subs(ics_n).subs(nums).subs(t,ti).n()
    print(round(ti, 3), x_a_ti, x_l_ti, v_l_ti)


0.0 0 0 0
0.056 126.393235405028 0.0213907875922671 1.07395816697702
0.111 219.536892123386 0.135440767714843 3.03457176559861
0.167 332.956589345882 0.347911462047590 4.45449095077648
0.222 484.802753513958 0.610920471703538 4.84970435073891
0.278 664.891792850549 0.872667364121343 4.49177441287424
0.333 853.147493807510 1.10607894231219 3.91606683179973
0.389 1034.59479825654 1.31133716661766 3.52191649093681
0.444 1204.74380468648 1.50311840523074 3.42905409724935
0.5 1367.04196832391 1.69625729292704 3.54554751136236


In [430]:
print(python(linsol_ics[2]))

T1 = Symbol('T1')
T2 = Symbol('T2')
omega_max = Symbol('omega_max')
v_l0 = Symbol('v_l0')
k = Symbol('k')
t = Symbol('t')
tau_max = Symbol('tau_max')
m_l = Symbol('m_l')
x_a0 = Symbol('x_a0')
x_l0 = Symbol('x_l0')
e = T1*T2*omega_max + 2*(-T1*T2*omega_max/2 + v_l0/2)*exp(-T1**2*k*omega_max*t/(2*tau_max))*cos(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max)) + k*(-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)*exp(-T1**2*k*omega_max*t/(2*tau_max))*sin(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(T2*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2))


In [431]:
e = T1*T2*omega_max + 2*(-T1*T2*omega_max/2 + v_l0/2)*exp(-T1**2*k*omega_max*t/(2*tau_max))*cos(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max)) + k*(-T1**3*T2**3*m_l*omega_max**2 + T1**2*T2**2*m_l*omega_max*v_l0 + 2*T1*T2*tau_max*x_a0 - 2*tau_max*x_l0)*exp(-T1**2*k*omega_max*t/(2*tau_max))*sin(t*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2)/(2*T2*m_l*tau_max))/(T2*sqrt(-T1**4*T2**2*k**2*m_l**2*omega_max**2 + 4*k*m_l*tau_max**2))
e.subs(nums)

(v_l0 - 3.75)*exp(-5.625*t)*cos(2.6353960802885*sqrt(30)*t) + 30.3559683488799*sqrt(30)*(0.00234375*v_l0 + 0.000125*x_a0 - 0.1*x_l0 - 0.0087890625)*exp(-5.625*t)*sin(2.6353960802885*sqrt(30)*t) + 3.75

In [432]:
print(pycode(code))

x0 = omega_max**2
x1 = m_l*x0
x2 = 1/tau_max
x3 = T1**2
x4 = T2**2
x5 = x2*x3*x4
x6 = T1*T2*omega_max
x7 = m_l*v_l0*x6 + tau_max*x_a0
x8 = (1/2)*v_l0 - 1/2*x6
x9 = 1/T2
x10 = math.sqrt(-T1**4*k**2*m_l**2*x0*x4 + 4*k*m_l*tau_max**2)
x11 = (1/2)*x2
x12 = t*x10*x11*x9/m_l
x13 = math.exp(-k*omega_max*t*x11*x3)
x14 = x13*math.cos(x12)
x15 = x14*x8
x16 = 2*x15
x17 = T1**3*T2**3*x1
x18 = 2*T1*T2*tau_max*x_a0 + m_l*omega_max*v_l0*x3*x4 - 2*tau_max*x_l0 - x17
x19 = math.sin(x12)
x20 = x13*x19/x10
x21 = x18*x20
x22 = k*m_l*omega_max
x23 = x11*x18
x_a_code = -T1*x2*x21*x22 - m_l*x16*x2*x6 + omega_max*t - x1*x5 + x2*x7
x_l_code = T1*T2*omega_max*t + T1*T2*x2*x7 - T2*x20*x22*x23*x3 + T2*x10*x13*x19*x2*x8/k - m_l*omega_max*x15*x5 - x14*x23 - x17*x2
v_l_code = k*x21*x9 + x16 + x6


In [438]:
import math

T1 = 0.00025 #0.035 m / 140 rad from TSA test
T2 = 5 #guess for leg movement/string contract
m_a = 1/2 * 0.020 * 0.010**2 #cylinder MOI 20g 10mm radius
m_l = 0.5 #mass to push off ground
k = 3000 # no idea honestly
tau_max = 0.05 #N*m stall torque
omega_max = 3000 #about 10,000rpm free speed

x_a0 = 0
x_l0 = 0
v_l0 = 0

t = 0.1

x0 = omega_max**2
x1 = m_l*x0
x2 = 1/tau_max
x3 = T1**2
x4 = T2**2
x5 = x2*x3*x4
x6 = T1*T2*omega_max
x7 = m_l*v_l0*x6 + tau_max*x_a0
x8 = (1/2)*v_l0 - 1/2*x6
x9 = 1/T2
x10 = math.sqrt(-T1**4*k**2*m_l**2*x0*x4 + 4*k*m_l*tau_max**2)
x11 = (1/2)*x2
x12 = t*x10*x11*x9/m_l
x13 = math.exp(-k*omega_max*t*x11*x3)
x14 = x13*math.cos(x12)
x15 = x14*x8
x16 = 2*x15
x17 = T1**3*T2**3*x1
x18 = 2*T1*T2*tau_max*x_a0 + m_l*omega_max*v_l0*x3*x4 - 2*tau_max*x_l0 - x17
x19 = math.sin(x12)
x20 = x13*x19/x10
x21 = x18*x20
x22 = k*m_l*omega_max
x23 = x11*x18
x_a_code = -T1*x2*x21*x22 - m_l*x16*x2*x6 + omega_max*t - x1*x5 + x2*x7
x_l_code = T1*T2*omega_max*t + T1*T2*x2*x7 - T2*x20*x22*x23*x3 + T2*x10*x13*x19*x2*x8/k - m_l*omega_max*x15*x5 - x14*x23 - x17*x2
v_l_code = k*x21*x9 + x16 + x6

print(x_a_code, x_l_code, v_l_code)

26.527713687383*v_l0 + 0.559521027193741*x_a0 + 352.383178245007*x_l0 + 200.521073672314 0.0723133285808962*v_l0 + 0.000884257122912769*x_a0 + 0.292594301669785*x_l0 + 0.103825017821639 0.292594301669785*v_l0 + 0.0117461059415002*x_a0 - 9.39688475320018*x_l0 + 2.6527713687383
